In [1]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [2]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [4]:
%cd nmt-kn-ml/

/content/nmt-kn-ml


In [6]:
! chmod a+x /content/nmt-kn-ml/scripts/data_preparation.sh

In [7]:
! /content/nmt-kn-ml/scripts/data_preparation.sh data.kn data.ml

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 2.54 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads:

In [8]:
! chmod a+x /content/nmt-kn-ml/scripts/split_dataset.sh

In [9]:
! /content/nmt-kn-ml/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.ml

In [10]:
%cd ..

/content


In [11]:
!fairseq-preprocess --source-lang kn --target-lang ml \
--trainpref nmt-kn-ml/train \
--validpref nmt-kn-ml/dev \
--testpref nmt-kn-ml/test \
--destdir nmt-kn-ml/tokenized.kn-ml \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-10 05:22:34 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 05:22:34 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-ml/tokenized.kn-ml', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [12]:
! fairseq-train nmt-kn-ml/tokenized.kn-ml \
--arch transformer \
--dropout 0.1 \
--attention-dropout 0.1 \
--activation-dropout 0.1 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 512 \
--encoder-layers 3 \
--encoder-attention-heads 8 \
--encoder-learned-pos \
--decoder-ffn-embed-dim 512 \
--decoder-layers 3 \
--decoder-attention-heads 8 \
--decoder-learned-pos \
--max-epoch 10 \
--optimizer adam \
--lr 5e-4 \
--batch-size 128 \
--seed 1 \
--wandb-project "Transformer from scratch - 10 March - kn to ml"

Streaming output truncated to the last 5000 lines.
|---------------------------------------------------------------------------|
| Allocated memory      |    8778 MB |    8846 MB |    2324 GB |    2316 GB |
|       from large pool |    8729 MB |    8797 MB |    2242 GB |    2234 GB |
|       from small pool |      49 MB |      60 MB |      82 GB |      81 GB |
|---------------------------------------------------------------------------|
| Active memory         |    8778 MB |    8846 MB |    2324 GB |    2316 GB |
|       from large pool |    8729 MB |    8797 MB |    2242 GB |    2234 GB |
|       from small pool |      49 MB |      60 MB |      82 GB |      81 GB |
|---------------------------------------------------------------------------|
| GPU reserved memory   |    9702 MB |   10864 MB |   58198 MB |   48496 MB |
|       from large pool |    9640 MB |   10730 MB |   56702 MB |   47062 MB |
|       from small pool |      62 MB |     134 MB |    1496 MB |    1434 MB |
|------------

In [14]:
!fairseq-generate /content/nmt-kn-ml/tokenized.kn-ml \
  --path /content/checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe \
 | grep ^H | LC_ALL=C sort -V | cut -f3- > test_ml_generated_scratch_transformer.txt 

2022-03-10 05:53:16 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 05:53:19 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_p

In [15]:
!fairseq-generate /content/nmt-kn-ml/tokenized.kn-ml \
  --path /content/checkpoints/checkpoint_best.pt \
  --beam 5 \
   --remove-bpe \
  | grep ^T | LC_ALL=C sort -V | cut -f2- > test_ml_actual_scratch_transformer.txt 

2022-03-10 05:53:49 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-10 05:53:52 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_p

In [16]:
!fairseq-score --sys /content/test_ml_generated_scratch_transformer.txt --ref /content/test_ml_actual_scratch_transformer.txt

2022-03-10 05:54:23 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
Namespace(ignore_case=False, order=4, ref='/content/test_ml_actual_scratch_transformer.txt', sacrebleu=False, sentence_bleu=False, sys='/content/test_ml_generated_scratch_transformer.txt')
BLEU4 = 14.36, 42.7/16.7/9.8/6.1 (BP=1.000, ratio=1.016, syslen=20324, reflen=20001)


In [17]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
f = open("/content/test_ml_generated_scratch_transformer.txt")
ml_candidates = [i.strip() for i in f.readlines()]
f = open("/content/test_ml_actual_scratch_transformer.txt")
ml_gold = [i.strip() for i in f.readlines()]
total_four = 0
for i in range(len(ml_candidates)):
    candidate = ml_candidates[i].split(" ")
    references = ml_gold[i].split(" ")
    reference = [references]
    score_cumulative4 = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
    total_four = total_four + score_cumulative4
print(total_four/len(ml_candidates))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.3430951762193223


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!cp /content/checkpoints/checkpoint_best.pt /content/drive/MyDrive/Transformer_from_scratch_ml/